In [1]:
#𝐕𝐢𝐫𝐭𝐮𝐚𝐥 𝐄𝐱𝐡𝐢𝐛𝐢𝐭𝐢𝐨𝐧 𝐆𝐞𝐧𝐞𝐫𝐚𝐭𝐨𝐫

# This project aims to create a virtual exhibition generator that uses the Spotify API 
# and the Art Institute of Chicago's API. The goal is to allow users to select art genres
# and receive an accompanying music playlist to enhance their viewing experience.

In [2]:
#Importing necessary tools
# Spotipy: For accessing the Spotify API.
# Pandas: For data handling and manipulation.
# Urllib: For making HTTP requests to APIs.
# IPython.display: For displaying images and media in the Notebook.

import spotipy
import spotipy.util as util
import json
import webbrowser
import pandas as pd
import urllib.request
from IPython.display import display, Image

In [3]:
#Accessing Spotify API 

cred = "spotify_keys.json"
with open(cred,"r") as key_file:
    api_tokens = json.load(key_file)

key_file.close()

In [4]:
client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username'] 

In [5]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirectURI)

In [6]:
sp = spotipy.Spotify(auth=token)

In [ ]:
# Available genre seeds from Spotify are fetched, which will later be used to 
# generate music recommendations based on the selected art categories.

genre_seeds = sp.recommendation_genre_seeds()

In [ ]:
genre_seeds

In [ ]:
# Artwork data is loaded from a CSV file of data from the AIC Github,containing information 
# about their most popular art pieces. This will help form the foundation for my virtual exhibition."


art_data = pd.read_csv("AIC_data.csv")

In [ ]:
display(art_data)

In [ ]:
# Unique department titles are extracted from the art data, providing insight into how 
# the artworks are categorised.

departments = art_data["department_title"].unique()

print("Department Titles",departments)

In [ ]:
# A request is made to the Art Institute API to retrieve additional artwork details, 
# including image IDs and classifications, which enhances the dataset I am working with.

url = "https://api.artic.edu/api/v1/artworks?fields=id,title,artist_display,date_display,classification_titles,department_title,image_id"
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)
all_art_data= json.loads(response.read())
all_art_data

In [ ]:
# Analysing artword data

for artwork in all_art_data['data'][:20]: 
    
    print("Title:", artwork['title'])
    print("Artist:", artwork.get('artist_display',))
    print("Date:", artwork.get('date_display'))
    print("Classification:", artwork.get('classification_titles'))
    print("Department:", artwork.get('department_title'))
    print("\n")

In [ ]:
#What are the unique department titles in this data set?
art_departments = [department_titles['department_title'] for department_titles in all_art_data['data']]
art_departments

In [ ]:
# I combine the artwork data from both the CSV and the API into a single DataFrame, 
# ensuring that all relevant information is consolidated for my code.

api_art_data = pd.DataFrame(data['data'])

csv_art_data = pd.read_csv("AIC_data.csv")

combined_art_data = pd.concat([csv_art_data, api_art_data], ignore_index=True)

In [ ]:
# I list all unique department titles in the combined dataset to get an overview of 
# the available art categories."

all_departments = combined_art_data["department_title"].unique()
print("All Departments:", all_departments)

all_classifications = combined_art_data["classification_titles"].explode().unique()
print("\nAll Classifications:", all_classifications)

In [ ]:
# The combined dataset is filtered to include only the selected departments relevant to 
# my exhibition, ensuring that only interesting artworks are considered.

selected_departments = ["Modern Art", "Contemporary Art", "Photography and Media", "Prints and Drawings", "Textiles"]

department_filter = combined_art_data["department_title"].isin(selected_departments)

filtered_art_data = combined_art_data[department_filter]

In [ ]:
filtered_art_data

In [ ]:
# Calculating the total number of artworks available in the filtered dataset.

artwork_count = len(filtered_art_data)
artwork_count

In [ ]:
# Counting the number of artworks in each selected department, 
# which provides insight into the distribution of artworks across categories.

department_counts = filtered_art_data["department_title"].value_counts()

department_counts

In [ ]:
# A list of artwork titles is created from the filtered dataset, and I print them out to quickly see the available artworks.

titles = filtered_art_data["title"].tolist()
for title in titles:
    print(title)

In [ ]:
# Art-to-Music Mapping Concept

# Modern Art
# Genre Seeds: ambient, experimental, trip-hop, electronic, idm
# Vibe: Abstract, innovative, and textured. These genres offer an atmospheric, boundary-pushing sound without being too familiar, 
# making them a perfect match for modern art that often explores the abstract and unconventional.

# Contemporary Art
# Genre Seeds: downtempo, synth-pop, chill, minimal-techno, new-age
# Vibe: Smooth and versatile, ideal for contemplative and bold contemporary pieces. These genres help create a modern, reflective atmosphere,
# supporting the dynamic and introspective qualities often found in contemporary art.

# Photography and Media
# Genre Seeds: dark-ambient, electronic, lo-fi, cinematic, ambient
# Vibe: Emotive and moody, with genres that add a cinematic or storytelling feel. These choices enhance the evocative quality of photography and media, 
# which often capture stories or moments frozen in time.

# Prints and Drawings
# Genre Seeds: ethereal, ambient, downtempo, soundscapes, neo-classical
# Vibe: Delicate and intricate, these genres enhance the tactile feel of prints and drawings. The music is intended to be immersive yet subtle, 
# echoing the careful details and craftsmanship of print and drawing techniques.

# Textiles
# Genre Seeds: world-music, new-age, ethereal, drone, ambient
# Vibe: Rooted yet expansive, offering a cultural or meditative quality without being overly traditional. These genres are intended to evoke a 
# connection to nature and tradition, while still feeling fresh and expansive for textile art.

In [ ]:
#Mapping what style of art should be matched with what genre of music.

art_to_music_map = {
    "Modern Art": ["ambient", "experimental", "trip-hop", "electronic", "idm"],
    "Contemporary Art": ["downtempo", "synth-pop", "chill", "minimal-techno", "new-age"],
    "Photography and Media": ["dark-ambient", "electronic", "lo-fi", "cinematic", "ambient"],
    "Prints and Drawings": ["ethereal", "ambient", "downtempo", "soundscapes", "neo-classical"],
    "Textiles": ["new-age", "ethereal", "drone", "ambient"]
}

In [ ]:
# Loop which generates and prints music recommendations for each art category based on 
# the genre mapping, demonstrating whcich music will compliment the art experience.

for art_category, genre_seeds in art_to_music_map.items():
    print(f"\nMusic Recommendations for {art_category}:")

    recommendations = sp.recommendations(seed_genres=genre_seeds, limit=5)

    for track in recommendations['tracks']:
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        print(f" ♫ {track_name} by {artist_name}")

In [ ]:
# At this point, I wanted to bring in images of the art pieces in my chosen categories so I can start creating randomised mixtures of the art
# and music. I know from the AIC API that they provide URLs for images and jupyter is able to show images. However, I realised at this point
# that in the CSV data set, there is no column for Image URLs. I tired getting all the image ids from my art data set and see if there is a 
# way for me to get URLs using those image ids. Unfortunately, that become too confusing and convoluted for me to understand so I pivioted 
# to collecting images straight from the API, still using my parameters. 

In [ ]:
# I defined the create_virtual_exhibition function, which takes filtered artwork data 
# and selected categories to display artworks and generate music recommendations.

# Inside this function, I retrieve artworks based on selected categories, display them, 
# fetch corresponding music recommendations, and create a Spotify playlist with the recommended tracks.
    

def create_virtual_exhibition(filtered_art_data, selected_categories, num_images=3, num_songs=5):
    track_uris = []
    print(f"Virtual Exhibition of {selected_categories}\n")

    for category in selected_categories:
        print(f"\nExhibition Category: {category}")

        # Getting artworks in the chosen category
        artworks_in_category = filtered_art_data[filtered_art_data["department_title"] == category]

        if artworks_in_category.empty:
            print(f"No artworks found for category: {category}")
            continue
        
        # Randomly select images from the category
        sample_artworks = artworks_in_category.sample(n=min(num_images, len(artworks_in_category)))
        for _, artwork in sample_artworks.iterrows():
            print(f"Title: {artwork['title']} - Artist: {artwork.get('artist_display', 'Unknown')}")
            image_url = artwork.get("image_url")
            if pd.notna(image_url) and image_url:
                display(Image(url=image_url, width=300))

        # Getting music recommendations based on the genre seeds for each category
        genre_seeds = art_to_music_map.get(category, [])
        recommendations = sp.recommendations(seed_genres=genre_seeds, limit=num_songs)
        for track in recommendations['tracks']:
            track_name = track['name']
            artist_name = track['artists'][0]['name']
            print(f" - {track_name} by {artist_name}")
            track_uris.append(track['uri'])

    # Create a Spotify playlist with the collected track URIs
    if track_uris:  # Check if there are any track URIs
        playlist = sp.user_playlist_create(user=username, name="Virtual Gallery Playlist", public=True)
        sp.playlist_add_items(playlist_id=playlist['id'], items=track_uris)
        print("\nSpotify Playlist Link:", playlist['external_urls']['spotify'])
    else:
        print("No songs to add to the playlist.")

In [ ]:
#"The exhibition function is called with specific categories, demonstrating how the 
# virtual exhibition works by displaying artworks and generating a music playlist that 
# aligns with those themes.

selected_categories = ["Modern Art", "Contemporary Art"]
create_virtual_exhibition(filtered_art_data, selected_categories, num_images=5, num_songs=5)

In [ ]:
selected_categories = ["Photography and Media", "Textiles"]
create_virtual_exhibition(filtered_art_data, selected_categories, num_images=5, num_songs=5)

In [ ]:
# In summary, this project successfully combines art and music to create an engaging 
# virtual exhibition experience. By structuring the code to load, filter, and display 
# artworks while generating tailored music recommendations, I aim to enhance the overall 
# enjoyment of the exhibition.